<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5> Classification with Python</font></h1>

#### This is for testing the Hidden comment
# @hidden_cell
username = arunay1@gmail.com
password = testpassword

In this notebook we practice all the classification algorithms that we learned so far in the course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

### Load Data From CSV File  

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object 

In [ ]:
df['due_date']       = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

In [ ]:
## In the Data Frame Columns Unnamed: 0 and Unnamed: 0.1 are repeated hence dropping one of them and Renaming the Columns for clarity
df = df.drop(['Unnamed: 0'],axis=1)
df.columns = ['ID','loan_status','Principal','terms','effective_date','due_date','age','education','Gender']
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:
df['loan_status'].replace(to_replace=['PAIDOFF','COLLECTION'], value=[0,1],inplace=True)
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

#### First lets Import Model evaluation Libraries

In [ ]:
from sklearn           import metrics
from sklearn.metrics   import classification_report,confusion_matrix, log_loss, f1_score

### Train Test Split  
Out of Sample Accuracy is the percentage of correct predictions that the model makes on data that that the model has NOT been trained on. Doing a train and test on the same dataset will most likely have low out-of-sample accuracy, due to the likelihood of being over-fit.

It is important that our models have a high, out-of-sample accuracy, because the purpose of any model, of course, is to make correct predictions on unknown data. So how can we improve out-of-sample accuracy? One way is to use an evaluation approach called Train/Test Split.
Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set. 

This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the data. It is more realistic for real world problems.

We use the same Train/Test data set splits in all the 4 classification models.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Training

We start the algorithm with different values of k to find the best K.

In [ ]:
Ks = 20
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):    
    #Train Model and Predict  
    clf_knn = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat_knn=clf_knn.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat_knn)
    
    std_acc[n-1]=np.std(yhat_knn==y_test)/np.sqrt(yhat_knn.shape[0])

mean_acc

### Accuracy evaluation and Selection of K
From above table we observe Accuracy is maximum for n=6 and n=7. But we select k=6 since that would be less prone to overfitting compared to k=7. Afterwards rerun the classifier and caluclate the accuracy.

In [ ]:
clf_knn   = KNeighborsClassifier(n_neighbors = 6).fit(X_train,y_train)
yhat_knn  = clf_knn.predict(X_test)

In [ ]:
print (classification_report(y_test, yhat_knn))

In [ ]:
print("KNN Train set Accuracy: ", metrics.accuracy_score(y_train, clf_knn.predict(X_train)))
print("KNN Test  set Accuracy: ", metrics.accuracy_score(y_test, yhat_knn))

# Decision Tree

First import the Decision Tree Algorithm

In [ ]:
from sklearn.tree import DecisionTreeClassifier

## Modeling
We will first create an instance of the <b>DecisionTreeClassifier</b> called <b>defaultTree</b>.<br> and we specify <i> criterion="entropy" </i> so we can see the information gain of each node.

In [ ]:
clf_dt = DecisionTreeClassifier(criterion="entropy", max_depth = 4)

#### We will fit the data with the training feature matrix X_train and training response vector y_train


In [ ]:
clf_dt.fit(X_train,y_train)

#### Prediction and Model Evaluation

In [ ]:
yhat_dt = clf_dt.predict(X_test)

In [ ]:
print("Decision Tree Train set Accuracy: ", metrics.accuracy_score(y_train, clf_dt.predict(X_train)))
print("Decision Tree Test  set Accuracy: ", metrics.accuracy_score(y_test, yhat_dt))

In [ ]:
print (classification_report(y_test, yhat_dt))

# Support Vector Machine

In [ ]:
from sklearn import svm

In [ ]:
clf_svm = svm.SVC(kernel='rbf')
clf_svm.fit(X_train, y_train) 

In [ ]:
yhat_svm = clf_svm.predict(X_test)

In [ ]:
print (classification_report(y_test, yhat_svm))

In [ ]:
f1_score(y_test, yhat_svm, average='weighted') 

In [ ]:
print("SVM Train set Accuracy: ", metrics.accuracy_score(y_train, clf_svm.predict(X_train)))
print("SVM Test  set Accuracy: ", metrics.accuracy_score(y_test, yhat_svm))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
clf_lr

In [ ]:
yhat_lr   = clf_lr.predict(X_test)
yhat_prob = clf_lr.predict_proba(X_test)

In [ ]:
print (classification_report(y_test, yhat_lr))

In [ ]:
log_loss(y_test, yhat_prob)

In [ ]:
print("Logistic Regression Train set Accuracy: ", metrics.accuracy_score(y_train, clf_lr.predict(X_train)))
print("Logistic Regression Test  set Accuracy: ", metrics.accuracy_score(y_test, yhat_lr))

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

#### Performing Same Preprocessing Steps as previously done on training Set.

In [ ]:
test_df = pd.read_csv('loan_test.csv')

### Removing un-necessary column
test_df = test_df.drop(['Unnamed: 0'],axis=1)
test_df.columns = ['ID','loan_status','Principal','terms','effective_date','due_date','age','education','Gender']

### Converting Due Date and effective Date to Pandas Date 
test_df['due_date']       = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])

### Adding Weekend 
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend']   = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)

## Transforming Gender
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)

### 
test_feature = test_df[['Principal','terms','age','Gender','weekend']]
test_feature = pd.concat([test_feature,pd.get_dummies(test_df['education'])], axis=1)
test_feature.drop(['Master or Above'], axis = 1,inplace=True)

X_TEST = test_feature
print(X_TEST[0:5])

test_df['loan_status'].replace(to_replace=['PAIDOFF','COLLECTION'], value=[0,1],inplace=True)
Y_TEST = test_df['loan_status'].values
print(Y_TEST[0:5])

### This is Important

<font color="brown"> While performing feature normalization on Test data, we perform feature normalisation over the training data. Then perform nomralisation on testing instances as well, but this time using the mean and variance of <b> Training explanatory variables </b>. In this way, we can test and evaluate whether our model can generalize well to new, unseen data points.
</font>

In [ ]:
X_TEST= preprocessing.StandardScaler().fit(X).transform(X_TEST)

In [ ]:
Yhat_TEST = clf_knn.predict(X_TEST)
print(metrics.accuracy_score(Y_TEST, Yhat_TEST))

Yhat_TEST = clf_dt.predict(X_TEST)
print(metrics.accuracy_score(Y_TEST, Yhat_TEST))

Yhat_TEST = clf_svm.predict(X_TEST)
print(metrics.accuracy_score(Y_TEST, Yhat_TEST))

Yhat_TEST = clf_lr.predict(X_TEST)
print(metrics.accuracy_score(Y_TEST, Yhat_TEST))



In [ ]:
classifiers = {"KNN": clf_knn,"Decision Tree": clf_dt, "SVM": clf_svm, "Logistic Regression": clf_lr }

summaryResult =pd.DataFrame()

print("           Algorithm :   Jaccard   :  F1-score  :   Accuracy : LogLoss")
for algo in classifiers:
    clf = classifiers[algo]
    Yhat_TEST = clf.predict(X_TEST)
    f1_score=0
    #f1_score= f1_score(Y_TEST, Yhat_TEST, average='weighted')    
    jaccard_score= jaccard_similarity_score(Y_TEST, Yhat_TEST)
    accuracy     = metrics.accuracy_score(Y_TEST, Yhat_TEST)
    
    if algo=="Decision Tree":        
        Yhat_prob = clf.predict_proba(X_TEST)
        logLoss = log_loss(Y_TEST, Yhat_prob)
    else:
        logLoss=0
        
    summaryRow = pd.Series([algo, jaccard_score,logLoss])    
    print("%20s :  %8.8f : %8.8f : %8.8f : %8.8f " %(algo,jaccard_score,f1_score,accuracy, logLoss ))
    

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>